<a href="https://colab.research.google.com/github/sqx27/AQS/blob/main/FM5091HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
#Alex Song FM5091 HW2


#S, the spot price of the asset at time t
#K, strike price of the option
#T, the maturity of the option. Time to maturity is defined as T−t
#N, the height of the binomial tree
#r, the risk-free interest rate, assumed to be constant between t and T
#q, the dividend yeild
#σ, volatility of underlying asset, the standard deviation of the asset returns

#calculate European Option prices using binomial tree
import numpy as np
import math

S = 100
K = 100
T = 1
N = 3
r = 0.06
q = 0
sigma = 0.2
#check the difference between European and American option approach, put option is more obvious
option_type = 'put' 

def European_binomial_tree(S,K,T,N,r,q,sigma,option_type):
  t = T/N
  u = math.exp(sigma*math.sqrt(t))
  d = 1/u
  p = (math.exp((r-q)*t)-d)/(u-d)
  discount = math.exp(-r*t)

  #create a matrix with size (N+1)*(N+1) to place all the price values. (0,0) position is the original price
  ptree = np.zeros([N+1,N+1])
  #place all the values inside the matrix
  for i in range(N+1):
    for j in range(i+1):
      ptree[j,i] = S*(d**j)*(u**(i-j))
  print(ptree)

  #create a second matrix to include all the value of options
  ovtree = np.zeros([N+1,N+1])
  #calculate the value of options at each point in the lattice
  if option_type == 'call':
    ovtree[:,N] = np.maximum(np.zeros(N+1), ptree[:,N]-K)
  else:
    ovtree[:,N] = np.maximum(np.zeros(N+1), K-ptree[:,N]) 

  #calculate the value back until we reach [0,0]
  for i in np.arange(N-1,-1,-1):
    for j in np.arange(0,i+1):
      ovtree[j,i] = discount * (p*ovtree[j,i+1] + (1-p)*ovtree[j+1,i+1])
  print(ovtree)
  delta = (ovtree[0,1]-ovtree[1,1])/(ptree[0,1]-ptree[1,1])
  #gamma = ((ovtree[0,2]-ovtree[1,2])/(ptree[0,2]-ptree[1,2])-(ovtree[1,2]-ovtree[2,2])/(ptree[1,2]-ptree[2,2])/(0.5*(ptree[0,2]-ptree[2,2]))

  return ovtree[0,0],delta;

print(European_binomial_tree(S,K,T,N,r,q,sigma,option_type))

[[100.         112.24009024 125.97837858 141.39824581]
 [  0.          89.09472523 100.         112.24009024]
 [  0.           0.          79.37870064  89.09472523]
 [  0.           0.           0.          70.72223522]]
[[ 5.72842654  2.04284228  0.          0.        ]
 [ 0.         10.65174591  4.71993182  0.        ]
 [ 0.          0.         18.64116669 10.90527477]
 [ 0.          0.          0.         29.27776478]]
(5.728426535389136, -0.37194935661830325)


In [39]:
#calculate American Option prices using binomial tree
S = 100
K = 100
T = 1
N = 3
r = 0.06
q = 0
sigma = 0.2
#check the difference between European and American option approach, put option is more obvious
option_type = 'put'

def American_binomial_tree(S,K,T,N,r,q,sigma,option_type):
  t = T/N
  u = math.exp(sigma*math.sqrt(t))
  d = 1/u
  p = (math.exp((r-q)*t)-d)/(u-d)
  discount = math.exp(-r*t)

  #create a matrix with size (N+1)*(N+1) to place all the price values. (0,0) position is the original price
  ptree = np.zeros([N+1,N+1])
  #place all the values inside the matrix
  for i in range(N+1):
    for j in range(i+1):
      ptree[j,i] = S*(d**j)*(u**(i-j))
  print(ptree)

  #create a second matrix to include all the value of options
  ovtree = np.zeros([N+1,N+1])
  #calculate the value of options at each point in the lattice, and calculate the value back until we reach [0,0]
  if option_type == 'call':
    ovtree[:,N] = np.maximum(np.zeros(N+1), ptree[:,N]-K)
    for i in np.arange(N-1,-1,-1):
      for j in np.arange(0,i+1):
        ovtree[j,i] = np.maximum(discount * (p*ovtree[j,i+1] + (1-p)*ovtree[j+1,i+1]), ptree[j,i]-K)
  else:
    ovtree[:,N] = np.maximum(np.zeros(N+1), K-ptree[:,N]) 
    for i in np.arange(N-1,-1,-1):
      for j in np.arange(0,i+1):
        ovtree[j,i] = np.maximum(discount * (p*ovtree[j,i+1] + (1-p)*ovtree[j+1,i+1]), K-ptree[j,i])
  print(ovtree)
  delta = (ovtree[0,1]-ovtree[1,1])/(ptree[0,1]-ptree[1,1])


  return ovtree[0,0], delta; 
print(American_binomial_tree(S,K,T,N,r,q,sigma,option_type))

[[100.         112.24009024 125.97837858 141.39824581]
 [  0.          89.09472523 100.         112.24009024]
 [  0.           0.          79.37870064  89.09472523]
 [  0.           0.           0.          70.72223522]]
[[ 6.09935701  2.04284228  0.          0.        ]
 [ 0.         11.50877073  4.71993182  0.        ]
 [ 0.          0.         20.62129936 10.90527477]
 [ 0.          0.          0.         29.27776478]]
(6.099357008892412, -0.4089772809232681)
